In [1]:
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import json

import unidecode
import simplejson

In [5]:
regions = gpd.read_file("../../data/regions.geojson")

regions["FILENAME"] = (
    regions["NAME_LATN"]
    .map(unidecode.unidecode)
    .str.replace(",", "")
    .str.replace(".", "")
    .str.replace(":", "")
    .str.replace("--", "")
    .str.replace("(", "")
    .str.replace(")", "")
    .str.replace("/", "")
    .str.replace("'", "")
)

regions = regions.set_index(regions["FILENAME"])
regions = regions.drop("Moldova")  # TODO

In [6]:
with open("../../data/metadata.json") as f:
    metadata = json.load(f)

## Merge single-value files, set region coordinate

In [54]:
%%bash
rm uncertainty/*/*/*/merged.nc

In [57]:
def add_region_dim(ds):
    name = os.path.basename(ds.encoding["source"])
    region = unidecode.unidecode(" ".join(name.split("_")[5:-2]))
    return ds.load().drop_vars(["time_bnds", "height"], errors="ignore").assign_coords({"region": [region]})

for variable in ["tas", "pr"]:
    for scenario in ["historical", "rcp26", "rcp45", "rcp85"]:
        for model in metadata["models"]:
            folder = f"uncertainty/{model['gcm']}_{model['rcm']}/{scenario}/{variable}"
            if not os.path.isdir(folder) or os.path.exists(f"{folder}/merged.nc"):
                print("skipping:", folder)
                continue
            print(folder)
            ds = xr.open_mfdataset(f"{folder}/*.nc", preprocess=add_region_dim)
            ds = ds.assign_coords({
                "scenario": [scenario],
                "model": ["{gcm} {rcm} {ens}".format(**model)]
            })
            ds["time"] = ds["time"].dt.year
            ds.to_netcdf(f"{folder}/merged.nc")
            ds.close()

skipping: uncertainty/CNRM-CERFACS-CNRM-CM5_CLMcom-CCLM4-8-17/historical/tas
skipping: uncertainty/CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63/historical/tas
skipping: uncertainty/CNRM-CERFACS-CNRM-CM5_KNMI-RACMO22E/historical/tas
skipping: uncertainty/CNRM-CERFACS-CNRM-CM5_SMHI-RCA4/historical/tas
skipping: uncertainty/ICHEC-EC-EARTH_CLMcom-CCLM4-8-17/historical/tas
skipping: uncertainty/ICHEC-EC-EARTH_DMI-HIRHAM5/historical/tas
skipping: uncertainty/ICHEC-EC-EARTH_GERICS-REMO2015/historical/tas
skipping: uncertainty/ICHEC-EC-EARTH_KNMI-RACMO22E/historical/tas
skipping: uncertainty/ICHEC-EC-EARTH_SMHI-RCA4/historical/tas
skipping: uncertainty/IPSL-IPSL-CM5A-MR_IPSL-WRF381P/historical/tas
skipping: uncertainty/IPSL-IPSL-CM5A-MR_SMHI-RCA4/historical/tas
skipping: uncertainty/MOHC-HadGEM2-ES_CLMcom-CCLM4-8-17/historical/tas
skipping: uncertainty/MOHC-HadGEM2-ES_DMI-HIRHAM5/historical/tas
skipping: uncertainty/MOHC-HadGEM2-ES_GERICS-REMO2015/historical/tas
skipping: uncertainty/MOHC-HadGEM2-ES_KN

## To JSON

In [2]:
ds_all = xr.open_mfdataset("uncertainty/*/*/*/merged.nc").squeeze().load()
ds_all.to_netcdf("uncertainty.nc")

In [8]:
set(ds_all.region.values) ^ set(regions["FILENAME"].values)

{'Area Metropolitana de Lisboa'}

In [9]:
proj = {}

for _, region in regions.iterrows():
    nuts_id = region["id"]
    filename = region["FILENAME"]

    proj[nuts_id] = {}
    if filename not in ds_all["region"]:
        continue

    for variable in ["tas", "pr"]:
        proj[nuts_id][variable] = []

        for model in metadata["models"]:
            model_str = "{gcm} {rcm} {ens}".format(**model)
            if model_str in ds_all["model"]:
                data = {}
                da = ds_all[variable].sel({
                    "region": filename,
                    "model": model_str
                }).astype(float)
                data["hist"] = da.sel({"scenario": "historical"}).dropna(dim="time").squeeze().values.round(3)

                for rcp in ["rcp26", "rcp45", "rcp85"]:
                    vals = da.sel({"scenario": rcp}).dropna(dim="time").squeeze().values.round(3).tolist()
                    if vals:
                        data[rcp] = vals
            else:
                data = None

            proj[nuts_id][variable].append(data)

In [10]:
def export(name, out):
    with open(f"../../data/{name}.json", "w") as f:
        simplejson.dump(out, f, sort_keys=True, ignore_nan=True)

In [11]:
export("projections", proj)